In [ ]:
from scipy import special
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## object class for node
class node():
    """the class for each node"""
    
    def __init__(self, left, right, dk):
        self.l = left
        self.r = right
        self.d = dk

In [ ]:
## function to calculate the probability of first hypothesis
def p_hyp1(dataset, a, b):
    # initiate probability
    p = 1
    
    # extract the number of features and the total number of data
    k = dataset.shape[1]
    N = dataset.shape[0]
    
    # iterate to calculate the probability
    for i in range(k):
        mi = sum(dataset[:,i])
        dom = special.gamma(a[i] + b[i]) * special.gamma(a[i] + mi) * special.gamma(b[i] + N - mi)
        denom = special.gamma(a[i]) * special.gamma(b[i]) * special.gamma(a[i] + b[i] + N)
        pi = dom / denom
        p = p * pi
    
    return p